In [36]:
import numpy as np
from numpy.fft import fft
from scipy import stats
from scipy.integrate import quad
import datetime

# **M1**

In [1]:
S0 = 100
K = 100
T = 1
r = 0.05
sigma = 0.2

## **Black-Scholes model**

In [32]:
# BS Model
def bs_analytic_call(S0, K, T, r, sigma):
    "BS model"
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    return S0 * stats.norm.cdf(d1, 0, 1) - np.exp(-r * T) * K * stats.norm.cdf(d2, 0, 1)

In [33]:
bs_analytic_call(S0, K, T, r, sigma).round(4)

10.4506

## **Fourier Transform as in Lewis**

### Black-Scholes Characteristics Function  

$$\varphi^{BS}(u,T)=e^{((r-\frac{\sigma^2}{2})iu-\frac{\sigma^2}{2} u^2)T}$$

### Intergrate in Lewis  

$$C_0 = S_0 - \frac{\sqrt{S_0K}e^{-rT}}{\pi}\int^\infty_0 Re [e^{izk}\varphi(z-i/2)] \frac{dz}{z^2 + 1/4}$$

In [24]:
def BS_characteristic_func(v, x0, T, r, sigma):
    """
    Computes general Black-Scholes model characteristic function
    to be used in Fourier pricing methods like Lewis (2001) and Carr-Madan (1999)
    """
    cf_values = np.exp(
        ((x0 / T + r - 0.5 * sigma**2) * 1j * v - 0.5 * sigma**2 * v**2) * T
    )
    return cf_values

In [25]:
def BS_integral(u, S0, K, T, r, sigma):
    """
    Expenssion for the integral in Lewis (2001)
    """
    cf_values = BS_characteristic_func(u - 1j * 0.5, 0.0, T, r, sigma)
    int_value = (
        1 / (u**2 + 0.25) * (np.exp(1j * u * (np.log(S0 / K))) * cf_values).real
    )
    return int_value


def BS_call_Lewis(S0, K, T, r, sigma):
    """
    European Call option price in BS under Lewis (2001)
    """
    int_value = quad(lambda u: BS_integral(u, S0, K, T, r, sigma), 0, 100)[0]
    call_value = max(0, S0 - np.exp(-r * T) * (np.sqrt(S0 * K)) / np.pi * int_value)
    return call_value

In [26]:
print(
    "Fourier Call Option price under Lewis (2001) is $",
    BS_call_Lewis(S0, K, T, r, sigma).round(4),
)

Fourier Call Option price under Lewis (2001) is $ 10.4506


### **Fast Fourier Transform (FFT) for Option Pricing**

In [27]:
def BS_call_fft(S0, K, T, r, sigma):
    """
    European Call Option price in BS under FFT
    """

    k = np.log(K / S0)
    x0 = np.log(S0 / S0)
    g = 1  # Factor to increase accuracy
    N = g * 4096
    eps = (g * 150) ** -1
    eta = 2 * np.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    vo = eta * (u - 1)

    # Modification to ensure integrability
    if S0 >= 0.95 * K:
        alpha = 1.5
        v = vo - (alpha + 1) * 1j
        modcharFunc = np.exp(-r * T) * (
            BS_characteristic_func(v, x0, T, r, sigma)
            / (alpha**2 + alpha - vo**2 + 1j * (2 * alpha + 1) * vo)
        )

    else:
        alpha = 1.1
        v = (vo - 1j * alpha) - 1j
        modcharFunc1 = np.exp(-r * T) * (
            1 / (1 + 1j * (vo - 1j * alpha))
            - np.exp(r * T) / (1j * (vo - 1j * alpha))
            - BS_characteristic_func(v, x0, T, r, sigma)
            / ((vo - 1j * alpha) ** 2 - 1j * (vo - 1j * alpha))
        )

        v = (vo + 1j * alpha) - 1j

        modcharFunc2 = np.exp(-r * T) * (
            1 / (1 + 1j * (vo + 1j * alpha))
            - np.exp(r * T) / (1j * (vo + 1j * alpha))
            - BS_characteristic_func(v, x0, T, r, sigma)
            / ((vo + 1j * alpha) ** 2 - 1j * (vo + 1j * alpha))
        )

    delt = np.zeros(N)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3
    if S0 >= 0.95 * K:
        FFTFunc = np.exp(1j * b * vo) * modcharFunc * eta * SimpsonW
        payoff = (fft(FFTFunc)).real
        CallValueM = np.exp(-alpha * k) / np.pi * payoff
    else:
        FFTFunc = (
            np.exp(1j * b * vo) * (modcharFunc1 - modcharFunc2) * 0.5 * eta * SimpsonW
        )
        payoff = (fft(FFTFunc)).real
        CallValueM = payoff / (np.sinh(alpha * k) * np.pi)

    pos = int((k + b) / eps)
    CallValue = CallValueM[pos] * S0

    return CallValue

In [28]:
BS_call_fft(S0, K, T, r, sigma).round(4)

10.4506

In [115]:
begin = datetime.datetime.utcnow()
bs_analytic_call(S0, K, T, r, sigma)
end = datetime.datetime.utcnow()
x = end - begin
print(x)

0:00:00.001001
